# Using AI and NLP to Improve Your Cooking

Michael Humkey, Conor Marsten

In [144]:
import urllib
import urllib.request

In [145]:
url = "http://food2fork.com/api/search?"
args = {'key' : 'd8b2df92a9cb994f2009b2be8410c1a3', 'q' : 'taco', 'sort' : 'r'}
data = urllib.parse.urlencode(args)
req = urllib.request.Request(url+data, headers={'User-Agent': 'Mozilla/5.0'})
response = urllib.request.urlopen(req).read()

In [146]:
import json

In [147]:
response = json.loads(response)
recipes = response['recipes']

In [148]:
from bs4 import BeautifulSoup
import requests

In [149]:
reviews, recipe = getReviews(recipes)    
# if type(reviews[0]) is not str:
revs = [rv.get_text().replace('\n', ' ').replace('\r', '') for rv in reviews]
# else:
#     revs = [rv.replace('\n', ' ').replace('\r', '') for rv in reviews]

In [150]:
def getReviews(recipes, i=0):

    recipe = recipes[i]
    url = "http://food2fork.com/api/get?"
    data = urllib.parse.urlencode({'key' : 'd8b2df92a9cb994f2009b2be8410c1a3', 'rId' : recipe['recipe_id']})
    req = urllib.request.Request(url+data, headers={'User-Agent': 'Mozilla/5.0'})
    recipe = json.loads(urllib.request.urlopen(req).read())['recipe']
    bs =  BeautifulSoup(requests.get(recipe['source_url']).content, 'html.parser')
    reviews = []

    if (recipe['publisher'] == '101 Cookbooks'): #1
        reviews = bs.find_all(attrs={'class': 'card-body'})

    elif (recipe['publisher'] == 'BBC Good Food'):
        reviews = bs.find_all(attrs={'class': 'field-item even'})

    elif (recipe['publisher'] == 'Closet Cooking' 
           or recipe['publisher'] == 'Eats Well With Others' 
           or recipe['publisher'] == "Lisa's Kitchen"
           or recipe['publisher'] == 'A Spicy Perspective'
           or recipe['publisher'] == 'Naturally Ella'
           or recipe['publisher'] == 'Pastry Affair'):
        reviews = bs.find_all(attrs={'class': 'comment-body'})

    elif (recipe['publisher'] == 'Food Republic'):
        reviews = bs.find_all(attrs={'class': '_5mdd'}) #uses facebook comments plugin

    elif (recipe['publisher'] == 'PBS Food'):
        reviews = bs.find_all(attrs={'class': 'post-message'})

    elif (recipe['publisher'] == 'Simply Recipes'
          or recipe['publisher'] == 'Homesick Texan'
          or recipe['publisher'] == 'Tasty Kitchen'):
        reviews = bs.find_all(attrs={'class': 'comment-text'})

    elif (recipe['publisher'] == 'Two Peas and Their Pod'
          or recipe['publisher'] == 'Cookie and Kate'
          or recipe['publisher'] == "Elana's Pantry"
          or recipe['publisher'] == 'My Baking Addiction'
          or recipe['publisher'] == 'Smitten Kitchen'
          or recipe['publisher'] == 'Vintage Mixer'
          or recipe['publisher'] == 'Cookin Canuck'
          or recipe['publisher'] == 'Healthy Delicious'
          or recipe['publisher'] == 'Steamy Kitchen'
          or recipe['publisher'] == "What's Gaby Cooking"
          or recipe['publisher'] == 'Bunky Cooks'
          or recipe['publisher'] == 'Serious Eats'):
        reviews = bs.find_all(attrs={'class': 'comment-content'})

    elif (recipe['publisher'] == 'All Recipes'):
#         reviews = bs.find_all(itemprop="reviewBody")
        reviews = bs.find_all(attrs={'class' : 'review-detail__link'})
        revList = []
        for review in reviews:
            revList.append(str(review).split(' ')[2].replace('href=\"', '').replace('\"', ''))
        reviews = []
        for url in revList:
            bs =  BeautifulSoup(requests.get(url).content, 'html.parser')
#             reviews.append(bs.find(itemprop="reviewBody").replace('<p itemprop="reviewBody">', '').replace('<p>', ''))
            reviews.append(bs.find(itemprop="reviewBody"))

    elif (recipe['publisher'] == 'Big Girls Small Kitchen'
          or recipe['publisher'] =='Jamie Oliver'
          or recipe['publisher'] == 'The Pioneer Woman'):
        reviews = bs.find_all(attrs={'data-role': 'message'})

    elif (recipe['publisher'] == 'Framed Cooks'
          or recipe['publisher'] == 'Picky Palate'):
        reviews = bs.find_all(attrs={'class': 'comment even thread-even depth-1'})

    elif (recipe['publisher'] == 'Bon Appetit'):
        reviews = bs.find_all(attrs={'class': 'review-body'})

    elif (recipe['publisher'] == 'Epicurious'):
        reviews = bs.find_all(attrs={'class': 'review-text'})

    elif (recipe['publisher'] == 'Cookstr'):
        reviews = bs.find_all(attrs={'class': 'commentText'})

    elif (recipe['publisher'] == 'Panini Happy'):
        #careful, sloppy html
        reviews = bs.find_all(attrs={'class': 'format_text'})

    elif (recipe['publisher'] == 'Real Simple'):
        reviews = bs.find_all(attrs={'class': 'comment_txt'})
        
    elif (recipe['publisher'] == 'Chow'):
        reviews = bs.find_all(itemprop="comment")

    elif (recipe['publisher'] == 'Delishhh'):
        reviews = bs.find_all(attrs={'class': 'commentmeta'})

    elif (recipe['publisher'] == 'Food Network'):
        reviews = bs.find_all(attrs={'class': 'gig-comment-body'})    
    else:
        print(recipe['publisher'])
        getReviews(recipes, i+1)
        
    return reviews, recipe

In [151]:
print(type(reviews[0]))
print(reviews[0])

<class 'bs4.element.Tag'>
<p itemprop="reviewBody">
                This is my recipe and I am thrilled you all are loving it!  Here is the recipe for Bill Echols Taco Seasoning.  It really makes this recipe.
1 tablespoon chili powder 
1/4 teaspoon garlic powder 
1/4 teaspoon onion powder 
1/4 teaspoon crushed red pepper flakes 
1/4 teaspoon dried oregano 
1/2 teaspoon paprika 
1 1/2 teaspoons ground cumin 
1 teaspoon sea salt 
1 teaspoon black pepper
            </p>


## Performing sentiment analysis

In [152]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as sia

In [153]:
analyzer = sia()
scored = []
for rev in revs:
    res = analyzer.polarity_scores(rev)
    tup = (rev, res)
    scored.append(tup)

In [154]:
def getKey(tup):
    return tup[1]['compound']

scored = list(reversed(sorted(scored, key=getKey)))

In [155]:
scored

[('                 I wasn\'t sure about making this as I hate beer and no one in my house drinks it.  I also don\'t like very spicy foods. However, I took a chance and used Heineken for this recipe as well as the taco seasoning and tomatoes with chilies and the soup was DELICIOUS. I couldn\'t detect a "beer" taste in the soup at all and found it to be a bit on the mild side. Everyone in my family loved it. I made another pot for tailgating - it was the most popular dish - everyone wanted seconds and the recipe. It\'s easy to put together.  You could easily substitute chicken broth, as many people have, and leave off the cheese and sour cream (or use reduced calorie products)and use baked chips to make it more diet friendly.  I put mine in the crock pot before going to bed.  The chicken was frozen when I put it in the pot.  The next morning, it was so well cooked, I didn\'t have to take it out and shred it.  All I had to do is "grab" each breast a couple of times with tongs and the chi

## Checking for foods

In [24]:
from nltk.corpus import wordnet as wn
food = wn.synset('food.n.02')

In [25]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///usda.sql3')
connection = engine.connect()

In [26]:
recipe['publisher']

'All Recipes'

In [27]:
recipe

{'f2f_url': 'http://food2fork.com/view/28924',
 'image_url': 'http://static.food2fork.com/9843414ab7.jpg',
 'ingredients': ['1 onion, chopped',
  '1 (16 ounce) can chili beans',
  '1 (15 ounce) can black beans',
  '1 (15 ounce) can whole kernel corn, drained',
  '1 (8 ounce) can tomato sauce',
  '1 (12 fluid ounce) can or bottle beer',
  '2 (10 ounce) cans diced tomatoes with green chilies, undrained',
  '1 (1.25 ounce) package taco seasoning',
  '3 whole skinless, boneless chicken breasts',
  'shredded Cheddar cheese (optional)',
  'sour cream (optional)',
  'crushed tortilla chips (optional)'],
 'publisher': 'All Recipes',
 'publisher_url': 'http://allrecipes.com',
 'recipe_id': '28924',
 'social_rank': 99.99999999999945,
 'source_url': 'http://allrecipes.com/Recipe/Slow-Cooker-Chicken-Taco-Soup/Detail.aspx',
 'title': 'Slow Cooker Chicken Taco Soup'}

In [158]:
import nltk
from nltk import word_tokenize, pos_tag
text = word_tokenize(reviews[0].get_text())
nltk.pos_tag(text)

[('This', 'DT'),
 ('is', 'VBZ'),
 ('my', 'PRP$'),
 ('recipe', 'NN'),
 ('and', 'CC'),
 ('I', 'PRP'),
 ('am', 'VBP'),
 ('thrilled', 'VBN'),
 ('you', 'PRP'),
 ('all', 'DT'),
 ('are', 'VBP'),
 ('loving', 'VBG'),
 ('it', 'PRP'),
 ('!', '.'),
 ('Here', 'RB'),
 ('is', 'VBZ'),
 ('the', 'DT'),
 ('recipe', 'NN'),
 ('for', 'IN'),
 ('Bill', 'NNP'),
 ('Echols', 'NNP'),
 ('Taco', 'NNP'),
 ('Seasoning', 'NNP'),
 ('.', '.'),
 ('It', 'PRP'),
 ('really', 'RB'),
 ('makes', 'VBZ'),
 ('this', 'DT'),
 ('recipe', 'NN'),
 ('.', '.'),
 ('1', 'CD'),
 ('tablespoon', 'NN'),
 ('chili', 'NN'),
 ('powder', 'NN'),
 ('1/4', 'CD'),
 ('teaspoon', 'NN'),
 ('garlic', 'JJ'),
 ('powder', 'NN'),
 ('1/4', 'CD'),
 ('teaspoon', 'NN'),
 ('onion', 'NN'),
 ('powder', 'NN'),
 ('1/4', 'CD'),
 ('teaspoon', 'NN'),
 ('crushed', 'VBD'),
 ('red', 'JJ'),
 ('pepper', 'NN'),
 ('flakes', 'VBZ'),
 ('1/4', 'CD'),
 ('teaspoon', 'NN'),
 ('dried', 'VBD'),
 ('oregano', 'JJ'),
 ('1/2', 'CD'),
 ('teaspoon', 'NN'),
 ('paprika', 'VBD'),
 ('1', 'CD'),
